# Data Cleaning Notebook
This notebook performs cleaning on market data for prediction.

In [20]:
from google.colab import drive
drive.mount('/content/drive')
import os

# Define the path to your MIDI files
table_path = '/content/drive/MyDrive/trading/transform_data/eventhub_flattened.csv'
df=pd.read_csv(table_path, parse_dates=['datetime'])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [22]:
df.head()

,requested_timestamp,symbol,name,error,datetime,open,high,low,close,volume,query_timestamp
0,2025-11-25 19:56 UTC,BTC-USD,Bitcoin / US Dollar,NaN,2025-11-25 19:56:00+00:00,86799.570312,86799.570312,86799.570312,86799.570312,93839360.0,2025-11-25 19:56
1,2025-11-25 19:56 UTC,ETH-USD,Ethereum / US Dollar,NaN,2025-11-25 19:56:00+00:00,2919.979492,2919.979492,2919.979492,2919.979492,2934784.0,2025-11-25 19:56
2,2025-11-25 19:56 UTC,GC=F,Gold Futures (GC=F),NaN,2025-11-25 19:56:00+00:00,4166.600098,4167.000000,4129.299805,4166.700195,87990.0,2025-11-25 19:56
3,2025-11-25 19:57 UTC,BTC-USD,Bitcoin / US Dollar,NaN,2025-11-25 19:56:00+00:00,86799.570312,86799.570312,86799.570312,86799.570312,93839360.0,2025-11-25 19:57
4,2025-11-25 19:57 UTC,ETH-USD,Ethereum / US Dollar,NaN,2025-11-25 19:56:00+00:00,2919.979492,2919.979492,2919.979492,2919.979492,2934784.0,2025-11-25 19:57


In [25]:
# 1. Separate rows that contain errors
error_rows = df[df['error'].notna() & (df['error'] != '')]

# 2. Keep only valid rows (rows without errors)
clean_df = df[df['error'].isna() | (df['error'] == '')].copy()

# 3. Drop the error column entirely
clean_df = clean_df.drop(columns=['error'])

clean_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 3258 entries, 0 to 3314
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   requested_timestamp  3258 non-null   object             
 1   symbol               3258 non-null   object             
 2   name                 3258 non-null   object             
 3   datetime             3258 non-null   datetime64[ns, UTC]
 4   open                 3258 non-null   float64            
 5   high                 3258 non-null   float64            
 6   low                  3258 non-null   float64            
 7   close                3258 non-null   float64            
 8   volume               3258 non-null   float64            
 9   query_timestamp      3258 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(5), object(4)
memory usage: 280.0+ KB


In [26]:
clean_df = clean_df.drop_duplicates(subset=['symbol','datetime'], keep='first')
clean_df.shape

(2842, 10)

In [27]:
clean_df['datetime'] = pd.to_datetime(clean_df['datetime'], utc=True, errors='coerce')
clean_df['requested_timestamp'] = pd.to_datetime(clean_df['requested_timestamp'], utc=True, errors='coerce')
clean_df['query_timestamp'] = pd.to_datetime(clean_df['query_timestamp'], utc=True, errors='coerce')
numeric_cols = ['open','high','low','close','volume']
clean_df[numeric_cols] = clean_df[numeric_cols].astype(float)
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2842 entries, 0 to 3314
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   requested_timestamp  2842 non-null   datetime64[ns, UTC]
 1   symbol               2842 non-null   object             
 2   name                 2842 non-null   object             
 3   datetime             2842 non-null   datetime64[ns, UTC]
 4   open                 2842 non-null   float64            
 5   high                 2842 non-null   float64            
 6   low                  2842 non-null   float64            
 7   close                2842 non-null   float64            
 8   volume               2842 non-null   float64            
 9   query_timestamp      2842 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](3), float64(5), object(2)
memory usage: 244.2+ KB


In [28]:
clean_df = clean_df.sort_values(by=['symbol','datetime'])
clean_df.head()

,requested_timestamp,symbol,name,datetime,open,high,low,close,volume,query_timestamp
0,2025-11-25 19:56:00+00:00,BTC-USD,Bitcoin / US Dollar,2025-11-25 19:56:00+00:00,86799.570312,86799.570312,86799.570312,86799.570312,93839360.0,2025-11-25 19:56:00+00:00
6,2025-11-25 19:58:00+00:00,BTC-USD,Bitcoin / US Dollar,2025-11-25 19:58:00+00:00,86800.156250,86800.156250,86800.156250,86800.156250,261066752.0,2025-11-25 19:58:00+00:00
9,2025-11-25 19:59:00+00:00,BTC-USD,Bitcoin / US Dollar,2025-11-25 19:59:00+00:00,86819.250000,86819.250000,86819.250000,86819.250000,0.0,2025-11-25 19:59:00+00:00
12,2025-11-25 20:00:00+00:00,BTC-USD,Bitcoin / US Dollar,2025-11-25 20:00:00+00:00,86848.335938,86848.335938,86848.335938,86848.335938,0.0,2025-11-25 20:00:00+00:00
18,2025-11-25 20:02:00+00:00,BTC-USD,Bitcoin / US Dollar,2025-11-25 20:02:00+00:00,86851.125000,86851.125000,86851.125000,86851.125000,0.0,2025-11-25 20:02:00+00:00


In [18]:
clean_df['return'] = clean_df.groupby('symbol')['close'].pct_change()
clean_df['rolling_mean_3'] = clean_df.groupby('symbol')['close'].rolling(3).mean().reset_index(level=0, drop=True)
clean_df['rolling_std_3'] = clean_df.groupby('symbol')['close'].rolling(3).std().reset_index(level=0, drop=True)
clean_df['close_lag_1'] = clean_df.groupby('symbol')['close'].shift(1)
clean_df['high_lag_1'] = clean_df.groupby('symbol')['high'].shift(1)
clean_df['low_lag_1'] = clean_df.groupby('symbol')['low'].shift(1)

In [34]:
import os

# Path to save the CSV
table_path = '/content/drive/MyDrive/trading/data/clean_data/cleaned_market_data.csv'

# Create the folder if it doesn't exist
os.makedirs(os.path.dirname(table_path), exist_ok=True)

# Save the cleaned DataFrame
clean_df.to_csv(table_path, index=False)

print(f"✅ Data saved successfully to {table_path}")


✅ Data saved successfully to /content/drive/MyDrive/trading/data/clean_data/cleaned_market_data.csv
